In [1]:
# Cell 1: Import libraries and load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
raw_data = pd.read_csv("../data/raw/data.csv", encoding="latin1")

print("✅ Data loaded successfully!")
print(f"📏 Dataset shape: {raw_data.shape}")
print(f"📋 Columns: {list(raw_data.columns)}")


✅ Data loaded successfully!
📏 Dataset shape: (541909, 8)
📋 Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


In [4]:

# Cell 3: First look at the data
print("=== FIRST 10 ROWS ===")
display(raw_data.head())

print("\n=== DATA TYPES ===")
print(raw_data.dtypes)

print("\n=== MISSING VALUES ===")
print(raw_data.isnull().sum())

print("\n=== BASIC STATISTICS ===")
display(raw_data.describe())

=== FIRST 10 ROWS ===


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom



=== DATA TYPES ===
InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

=== MISSING VALUES ===
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

=== BASIC STATISTICS ===


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
# Cell 4: Set up MySQL connection
import pymysql
from sqlalchemy import create_engine

# MySQL connection details
mysql_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Ivyyepal08!', 
    'database': 'retail_analytics'
}

# Create connection
try:
    connection_string = f"mysql+pymysql://{mysql_config['user']}:{mysql_config['password']}@{mysql_config['host']}/{mysql_config['database']}"
    engine = create_engine(connection_string)
    print("✅ MySQL connection created successfully!")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("We'll continue with local analysis for now")

✅ MySQL connection created successfully!


In [7]:
# Cell 5: Create cleaning log and start cleaning
print("=== STARTING DATA CLEANING ===")

# Create a copy for cleaning (always preserve original!)
clean_data = raw_data.copy()

# Log initial state
cleaning_log = {
    'timestamp': datetime.now(),
    'initial_rows': len(raw_data),
    'initial_columns': len(raw_data.columns),
    'steps_completed': []
}

print(f"📊 Starting with {len(clean_data)} rows and {len(clean_data.columns)} columns")

=== STARTING DATA CLEANING ===
📊 Starting with 541909 rows and 8 columns


In [8]:
# Cell 6: Normalize column names
print("=== STEP 1: NORMALIZING COLUMN NAMES ===")

# Show original column names
print("Original columns:", list(clean_data.columns))

# Normalize: lowercase, replace spaces with underscores
clean_data.columns = clean_data.columns.str.lower().str.replace(' ', '_').str.replace('.', '_')

print("Cleaned columns:", list(clean_data.columns))
cleaning_log['steps_completed'].append('Column names normalized')

=== STEP 1: NORMALIZING COLUMN NAMES ===
Original columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
Cleaned columns: ['invoiceno', 'stockcode', 'description', 'quantity', 'invoicedate', 'unitprice', 'customerid', 'country']


In [9]:
# Cell 7: Parse dates (we'll adapt this based on your data structure)
print("=== STEP 2: PARSING DATES ===")

# First, let's see what date columns we have
date_columns = clean_data.select_dtypes(include=['object']).columns
print("Potential date columns:", list(date_columns))

# We'll update this based on your actual data structure
# For now, let's identify date patterns
for col in date_columns[:3]:  # Check first 3 text columns
    print(f"\n{col} sample values:")
    print(clean_data[col].head())

=== STEP 2: PARSING DATES ===
Potential date columns: ['invoiceno', 'stockcode', 'description', 'invoicedate', 'country']

invoiceno sample values:
0    536365
1    536365
2    536365
3    536365
4    536365
Name: invoiceno, dtype: object

stockcode sample values:
0    85123A
1     71053
2    84406B
3    84029G
4    84029E
Name: stockcode, dtype: object

description sample values:
0     WHITE HANGING HEART T-LIGHT HOLDER
1                    WHITE METAL LANTERN
2         CREAM CUPID HEARTS COAT HANGER
3    KNITTED UNION FLAG HOT WATER BOTTLE
4         RED WOOLLY HOTTIE WHITE HEART.
Name: description, dtype: object


In [10]:
# Cell 8: Parse the invoice date properly
print("=== STEP 2: PARSING DATES (CONTINUED) ===")

# Let's look at the invoicedate column specifically
print("InvoiceDate sample values:")
print(clean_data['invoicedate'].head(10))
print(f"\nInvoiceDate data type: {clean_data['invoicedate'].dtype}")

# Parse the date column
try:
    clean_data['invoicedate'] = pd.to_datetime(clean_data['invoicedate'])
    print("✅ Date parsing successful!")
    
    # Show date range
    print(f"📅 Date range: {clean_data['invoicedate'].min()} to {clean_data['invoicedate'].max()}")
    
    cleaning_log['steps_completed'].append('Dates parsed successfully')
except Exception as e:
    print(f"❌ Date parsing failed: {e}")
    print("Let's examine the date format more closely...")

=== STEP 2: PARSING DATES (CONTINUED) ===
InvoiceDate sample values:
0    12/1/2010 8:26
1    12/1/2010 8:26
2    12/1/2010 8:26
3    12/1/2010 8:26
4    12/1/2010 8:26
5    12/1/2010 8:26
6    12/1/2010 8:26
7    12/1/2010 8:28
8    12/1/2010 8:28
9    12/1/2010 8:34
Name: invoicedate, dtype: object

InvoiceDate data type: object
✅ Date parsing successful!
📅 Date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00


In [11]:
# Cell 9: Check and remove duplicates
print("=== STEP 3: HANDLING DUPLICATES ===")

initial_rows = len(clean_data)
print(f"📊 Initial row count: {initial_rows}")

# Check for exact duplicates
duplicates = clean_data.duplicated().sum()
print(f"🔍 Found {duplicates} exact duplicate rows")

# Remove duplicates
clean_data = clean_data.drop_duplicates()
final_rows = len(clean_data)

print(f"📊 After removing duplicates: {final_rows} rows")
print(f"🗑️ Removed {initial_rows - final_rows} duplicate rows")

cleaning_log['steps_completed'].append(f'Removed {initial_rows - final_rows} duplicates')
cleaning_log['rows_after_deduplication'] = final_rows

=== STEP 3: HANDLING DUPLICATES ===
📊 Initial row count: 541909
🔍 Found 5268 exact duplicate rows
📊 After removing duplicates: 536641 rows
🗑️ Removed 5268 duplicate rows


In [12]:
# Cell 10: Examine and handle missing values
print("=== STEP 4: HANDLING MISSING VALUES ===")

print("Missing values per column:")
missing_summary = clean_data.isnull().sum()
print(missing_summary)

# Show percentage of missing values
print("\nMissing values percentage:")
missing_percent = (missing_summary / len(clean_data)) * 100
for col, pct in missing_percent.items():
    if pct > 0:
        print(f"{col}: {pct:.2f}%")

# Handle missing CustomerID (common in this dataset)
if 'customerid' in clean_data.columns:
    missing_customers = clean_data['customerid'].isnull().sum()
    print(f"\n🔍 Missing CustomerID: {missing_customers} rows ({missing_customers/len(clean_data)*100:.1f}%)")
    
    # For customer segmentation, we'll remove rows without CustomerID
    clean_data_with_customers = clean_data.dropna(subset=['customerid'])
    print(f"📊 Rows with valid CustomerID: {len(clean_data_with_customers)}")
    
    # Update our working dataset
    clean_data = clean_data_with_customers
    cleaning_log['steps_completed'].append(f'Removed {missing_customers} rows with missing CustomerID')

=== STEP 4: HANDLING MISSING VALUES ===
Missing values per column:
invoiceno           0
stockcode           0
description      1454
quantity            0
invoicedate         0
unitprice           0
customerid     135037
country             0
dtype: int64

Missing values percentage:
description: 0.27%
customerid: 25.16%

🔍 Missing CustomerID: 135037 rows (25.2%)
📊 Rows with valid CustomerID: 401604


In [13]:
# Cell 11: Fix negative quantities and prices
print("=== STEP 5: HANDLING INVALID VALUES ===")

# Check for negative quantities
negative_qty = (clean_data['quantity'] <= 0).sum()
print(f"🔍 Rows with negative/zero quantity: {negative_qty}")

# Check for negative prices
negative_price = (clean_data['unitprice'] <= 0).sum()
print(f"🔍 Rows with negative/zero unit price: {negative_price}")

# Show some examples of negative values
if negative_qty > 0:
    print("\nSample negative quantities:")
    print(clean_data[clean_data['quantity'] <= 0][['invoiceno', 'stockcode', 'quantity', 'description']].head())

# Remove rows with negative/zero quantities or prices
before_cleaning = len(clean_data)
clean_data = clean_data[(clean_data['quantity'] > 0) & (clean_data['unitprice'] > 0)]
after_cleaning = len(clean_data)

print(f"\n📊 Removed {before_cleaning - after_cleaning} rows with invalid quantities/prices")
print(f"📊 Final dataset: {after_cleaning} rows")

cleaning_log['steps_completed'].append(f'Removed {before_cleaning - after_cleaning} rows with invalid values')
cleaning_log['final_clean_rows'] = after_cleaning

=== STEP 5: HANDLING INVALID VALUES ===
🔍 Rows with negative/zero quantity: 8872
🔍 Rows with negative/zero unit price: 40

Sample negative quantities:
    invoiceno stockcode  quantity                       description
141   C536379         D        -1                          Discount
154   C536383    35004C        -1   SET OF 3 COLOURED  FLYING DUCKS
235   C536391     22556       -12    PLASTERS IN TIN CIRCUS PARADE 
236   C536391     21984       -24  PACK OF 12 PINK PAISLEY TISSUES 
237   C536391     21983       -24  PACK OF 12 BLUE PAISLEY TISSUES 

📊 Removed 8912 rows with invalid quantities/prices
📊 Final dataset: 392692 rows


In [21]:
# Cell 12: Create useful features for analysis
print("=== STEP 6: FEATURE ENGINEERING ===")

# Create total amount per transaction line
clean_data['total_amount'] = clean_data['quantity'] * clean_data['unitprice']

# Extract date components for analysis
clean_data['year'] = clean_data['invoicedate'].dt.year
clean_data['month'] = clean_data['invoicedate'].dt.month
clean_data['day_of_week'] = clean_data['invoicedate'].dt.dayofweek
clean_data['hour'] = clean_data['invoicedate'].dt.hour

print("✅ Created new features:")
print("- total_amount (quantity × unit_price)")
print("- year, month, day_of_week, hour")

# Show sample of enhanced data
print("\nSample of enhanced data:")
display(clean_data[['invoiceno', 'quantity', 'unitprice', 'total_amount', 'year', 'month']].head())

cleaning_log['steps_completed'].append('Created additional features')

=== STEP 6: FEATURE ENGINEERING ===
✅ Created new features:
- total_amount (quantity × unit_price)
- year, month, day_of_week, hour

Sample of enhanced data:


,invoiceno,quantity,unitprice,total_amount,year,month
0,536365,6,2.55,15.30,2010,12
1,536365,6,3.39,20.34,2010,12
2,536365,8,2.75,22.00,2010,12
3,536365,6,3.39,20.34,2010,12
4,536365,6,3.39,20.34,2010,12


In [22]:
# Cell 13: Final validation and summary
print("=== STEP 7: FINAL VALIDATION ===")

print("📊 FINAL DATASET SUMMARY:")
print(f"Rows: {len(clean_data):,}")
print(f"Columns: {len(clean_data.columns)}")
print(f"Date range: {clean_data['invoicedate'].min()} to {clean_data['invoicedate'].max()}")
print(f"Unique customers: {clean_data['customerid'].nunique():,}")
print(f"Unique products: {clean_data['stockcode'].nunique():,}")
print(f"Countries: {clean_data['country'].nunique()}")

print("\n💰 REVENUE SUMMARY:")
total_revenue = clean_data['total_amount'].sum()
print(f"Total revenue: £{total_revenue:,.2f}")
print(f"Average order value: £{clean_data['total_amount'].mean():.2f}")

print("\n📈 TOP 5 COUNTRIES BY REVENUE:")
country_revenue = clean_data.groupby('country')['total_amount'].sum().sort_values(ascending=False).head()
display(country_revenue)

# Complete the cleaning log
cleaning_log['completion_time'] = datetime.now()
cleaning_log['processing_duration'] = cleaning_log['completion_time'] - cleaning_log['timestamp']

print("\n📝 CLEANING LOG:")
for step in cleaning_log['steps_completed']:
    print(f"✅ {step}")

=== STEP 7: FINAL VALIDATION ===
📊 FINAL DATASET SUMMARY:
Rows: 392,692
Columns: 13
Date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Unique customers: 4,338
Unique products: 3,665
Countries: 37

💰 REVENUE SUMMARY:
Total revenue: £8,887,208.89
Average order value: £22.63

📈 TOP 5 COUNTRIES BY REVENUE:


country
United Kingdom    7285024.644
Netherlands        285446.340
EIRE               265262.460
Germany            228678.400
France             208934.310
Name: total_amount, dtype: float64


📝 CLEANING LOG:
✅ Column names normalized
✅ Dates parsed successfully
✅ Removed 5268 duplicates
✅ Removed 135037 rows with missing CustomerID
✅ Removed 8912 rows with invalid values
✅ Created additional features


In [27]:
# Cell 14: Save the cleaned data
print("=== STEP 8: SAVING CLEAN DATA ===")

# Save to CSV
csv_path = '../data/clean_sales.csv'
clean_data.to_csv(csv_path, index=False)
print(f"✅ Saved clean data to {csv_path}")

# Create a data summary file
summary_data = {
    'cleaning_timestamp': cleaning_log['timestamp'].strftime('%Y-%m-%d %H:%M:%S'),
    'original_rows': cleaning_log['initial_rows'],
    'final_rows': cleaning_log['final_clean_rows'],
    'data_quality_score': round((cleaning_log['final_clean_rows'] / cleaning_log['initial_rows']) * 100, 2),
    'date_range_start': str(clean_data['invoicedate'].min()),
    'date_range_end': str(clean_data['invoicedate'].max()),
    'unique_customers': int(clean_data['customerid'].nunique()),
    'unique_products': int(clean_data['stockcode'].nunique()),
    'total_revenue': float(clean_data['total_amount'].sum()),
    'countries': int(clean_data['country'].nunique())
}

# Save summary as JSON
import json
with open('../data/cleaning_summary.json', 'w') as f:
    json.dump(summary_data, f, indent=2)

print("✅ Saved cleaning summary to data/cleaning_summary.json")
print(f"📊 Data quality score: {summary_data['data_quality_score']}%")

=== STEP 8: SAVING CLEAN DATA ===
✅ Saved clean data to ../data/clean_sales.csv
✅ Saved cleaning summary to data/cleaning_summary.json
📊 Data quality score: 72.46%


In [28]:
# Cell 15: Display final structure for commit
print("=== READY FOR GIT COMMIT ===")
print("Files created:")
print("✅ data/clean_sales.csv")
print("✅ data/cleaning_summary.json") 
print("✅ notebooks/01_data_prep.ipynb")

print("\nYour Day 1 objectives are complete! 🎉")

=== READY FOR GIT COMMIT ===
Files created:
✅ data/clean_sales.csv
✅ data/cleaning_summary.json
✅ notebooks/01_data_prep.ipynb

Your Day 1 objectives are complete! 🎉
